# ฺBiGRU Embedding

In [ ]:
#BiGRU Embedding
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import GRU, Bidirectional, LSTM, Dense, Dropout, Input, BatchNormalization, Attention, Layer
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Enable TensorFlow's eager execution
tf.config.experimental_run_functions_eagerly(True)

file_paths = [
    "Data/Fused_Feature_Train.csv",
]

# Read and preprocess CSV files
df = pd.read_csv(file_paths[0])
df = df.iloc[:, 1:-1]  # Remove the last column

# Custom Mean Reduction Layer
class MeanReduction(Layer):
    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=1)

# Custom Reshape Layer
class ReshapeLayer(Layer):
    def __init__(self, new_shape):
        super(ReshapeLayer, self).__init__()
        self.new_shape = new_shape

    def call(self, inputs):
        return tf.reshape(inputs, self.new_shape)

# Function to create and predict models
def create_and_predict(model_type, Dim_value, X):
    input_shape = (X.shape[1], X.shape[2])
    inputs = Input(shape=input_shape)
    x = Bidirectional(GRU(units=int(Dim_value / 2), return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(inputs)
   
    # Attention layer
    attention = Attention()([x, x])
    attention = MeanReduction()(attention)  # Custom layer to reduce mean
    
    # Ensure fully-defined shape before BatchNormalization
    new_shape = [-1, attention.shape[-1]]  # Shape to reshape attention output
    attention = ReshapeLayer(new_shape=new_shape)(attention)  # Using custom reshape layer
    
    x = BatchNormalization()(attention)
    x = Dropout(0.3)(x)  # Adjusted dropout rate
    outputs = Dense(X.shape[2], activation='linear')(x)
    
    model = Model(inputs, outputs)
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=1e-4))  # Smaller learning rate
    
    # Early stopping and learning rate reduction callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    
    # Train the model with early stopping
    model.fit(X, X, epochs=1000, batch_size=16, verbose=1, validation_split=0.2, callbacks=[early_stopping, reduce_lr])
    
    # Extract features using the encoder part of the autoencoder
    encoder_model = Model(inputs, attention)  # Create a model from inputs to the attention layer
    features = encoder_model.predict(X)
    
    # Save features to CSV
    data_csv = pd.DataFrame(data=features)
    new_column_names = [f'{model_type}-{Dim_value}_{i}' for i in range(features.shape[1])]
    data_csv.columns = new_column_names
    data_csv.to_csv(f'{model_type}-{Dim_value}_train.csv', index=True)

# Preprocess data
data = df.apply(pd.to_numeric, errors='coerce')
data.dropna(inplace=True)
data_scaled = StandardScaler().fit_transform(data.values)
X = np.reshape(data_scaled, (data_scaled.shape[0], 1, data_scaled.shape[1]))

# Call the models 
create_and_predict('BiGRU', 4000, X)

